In [1]:
import matplotlib.pyplot as plt
#plt.style.use('classic')
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import matplotlib.patheffects as mpe
import matplotlib
from imblearn.over_sampling import SMOTE

In [2]:
df = pd.read_csv("ENC_29 - Sheet13.csv")

In [3]:
df.columns

Index(['sr#', 'ID', 'ExpID', 'Algo', 'timeslot_sec', 'timeslot_mil',
       'InterIntervalTime', 'InterIntervalTime_gt100', 'AveragePTime',
       'AveragePTime_gt100', 'TP', 'Total_Packets', 'Total_Packets_gt100',
       'STD', 'Min', 'Max', 'Width_Height', 'P1203', 'Stall', 'Bitrate',
       'P1203C', 'P1203_2C'],
      dtype='object')

In [4]:
df =df[(df['Algo']=='conventional') | (df['Algo']=='elastic')]

In [5]:
df.shape[0]

4126

In [6]:
df['P1203_2C'].unique()

array(['Bad', 'Fair', 'Good', 'Excellent'], dtype=object)

In [7]:
df['STD']= df['STD'].fillna(value=df['STD'].mean())

In [8]:
# # df.loc[df['Width_Height'] == '320x180', 'Width_Height'] = 1
# # cond16 = (df['daychange']<8) & (df['daychange']>2)
# def Classification(CLASS):
#     if CLASS == 's1-eth2':
#         return 1
#     elif CLASS == 's1-eth1':
#         return 'S1'
#     elif CLASS == 's2-eth2':
#         return 3
#     elif CLASS == 's2-eth1':
#         return 'S2'
#     elif CLASS == 's3-eth2':
#         return 5
#     elif CLASS == 's3-eth1':
#         return 6
#     elif CLASS == 's8-eth1':
#         return 7

In [9]:
# df['LR']=df['interface'].apply(Classification)

In [10]:
from sklearn import preprocessing


In [11]:
X= df.iloc[:, 6:14]

In [12]:
# X=preprocessing.normalize(X)
X= preprocessing.scale(X)

In [13]:
y = df.iloc[:, 21]

In [14]:
y

0             Bad
1             Bad
2             Bad
3            Fair
4            Fair
          ...    
4855    Excellent
4856    Excellent
4857    Excellent
4858    Excellent
4859    Excellent
Name: P1203_2C, Length: 4126, dtype: object

In [29]:
X.shape[0]

7228

In [30]:
X

array([[-0.58372688, -0.48827724, -0.34185115, ..., -0.33935434,
        -0.62420632, -0.49275414],
       [-1.38293862, -1.27529154, -0.57806316, ..., -0.22869998,
        -0.24690259, -0.47322186],
       [-0.96621199, -0.86326379, -0.53953537, ..., -0.05165301,
        -0.06934789,  1.51688551],
       ...,
       [ 1.12071963,  1.23447539, -0.55248824, ...,  1.65531836,
         1.73129594,  0.90677764],
       [-1.5487811 , -1.42369655, -0.58374722, ..., -0.34792551,
        -0.34183796, -0.22384949],
       [ 1.10810559,  1.22237475, -0.56334687, ...,  1.80188509,
         1.87828298,  0.33075788]])

In [31]:
sm = SMOTE()
X, y = sm.fit_sample(X, y)

In [32]:
X.shape[0]

7228

In [33]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, stratify=y, random_state = 42)

In [34]:
# Fitting Classifier to the Training Set
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(criterion="entropy", max_depth=5)
classifier.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=5)

In [35]:
dt1_score_train = classifier.score(X_train, y_train)
print("Training score: ", dt1_score_train)
dt1_score_test = classifier.score(X_test, y_test)
print("Testing score: ", dt1_score_test)


Training score:  0.6115833168610397
Testing score:  0.5984324573536192


In [36]:
# Model performance on training set
y_pred_train =classifier.predict(X_train)

from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report

accuracy = metrics.accuracy_score(y_train, y_pred_train)
print("Accuracy: {:.2f}".format(accuracy))
cm=confusion_matrix(y_train,y_pred_train)
print('Confusion Matrix: \n', cm)
print(classification_report(y_train, y_pred_train))

Accuracy: 0.61
Confusion Matrix: 
 [[748 224 282  11]
 [ 66 757  61 381]
 [207 189 739 129]
 [ 22 290 103 850]]
              precision    recall  f1-score   support

         Bad       0.72      0.59      0.65      1265
   Excellent       0.52      0.60      0.56      1265
        Fair       0.62      0.58      0.60      1264
        Good       0.62      0.67      0.64      1265

    accuracy                           0.61      5059
   macro avg       0.62      0.61      0.61      5059
weighted avg       0.62      0.61      0.61      5059



In [37]:
#Model Performance on Testing Data
y_pred_test =classifier.predict(X_test)

accuracy = metrics.accuracy_score(y_test, y_pred_test)
print("Accuracy: {:.2f}".format(accuracy))
cm=confusion_matrix(y_test,y_pred_test)
print('Confusion Matrix: \n', cm)
print(classification_report(y_test, y_pred_test))

Accuracy: 0.60
Confusion Matrix: 
 [[325  93 120   4]
 [ 27 313  24 178]
 [ 99  91 303  50]
 [ 10 131  44 357]]
              precision    recall  f1-score   support

         Bad       0.70      0.60      0.65       542
   Excellent       0.50      0.58      0.54       542
        Fair       0.62      0.56      0.59       543
        Good       0.61      0.66      0.63       542

    accuracy                           0.60      2169
   macro avg       0.61      0.60      0.60      2169
weighted avg       0.61      0.60      0.60      2169



In [38]:
print(classifier.tree_.max_depth)

5


In [39]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 6)

In [40]:
knn.fit(X_train,y_train)

KNeighborsClassifier(n_neighbors=6)

In [41]:
knn.score(X_test,y_test)

0.7026279391424619

In [42]:
y_pred = knn.predict(X_test)

In [43]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.7026279391424619


In [44]:
cm=confusion_matrix(y_test,y_pred)
print('Confusion Matrix: \n', cm)
print(classification_report(y_test, y_pred))

Confusion Matrix: 
 [[385  57  88  12]
 [ 42 341  42 117]
 [ 75  34 412  22]
 [ 10 106  40 386]]
              precision    recall  f1-score   support

         Bad       0.75      0.71      0.73       542
   Excellent       0.63      0.63      0.63       542
        Fair       0.71      0.76      0.73       543
        Good       0.72      0.71      0.72       542

    accuracy                           0.70      2169
   macro avg       0.70      0.70      0.70      2169
weighted avg       0.70      0.70      0.70      2169



In [45]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
rf = RandomForestClassifier(n_estimators=300)
rf.fit(X_train, y_train)

RandomForestClassifier(n_estimators=300)

In [46]:
rf_predictions = rf.predict(X_test)
accuracy_score(y_test, rf_predictions)
print('Accuracy is ' + str(round(accuracy_score(y_test, rf_predictions),2)))

Accuracy is 0.78


In [47]:
print(classification_report(y_test, rf_predictions))

              precision    recall  f1-score   support

         Bad       0.79      0.78      0.79       542
   Excellent       0.73      0.72      0.72       542
        Fair       0.81      0.78      0.80       543
        Good       0.78      0.82      0.80       542

    accuracy                           0.78      2169
   macro avg       0.78      0.78      0.78      2169
weighted avg       0.78      0.78      0.78      2169



In [48]:
cm = confusion_matrix(y_test, rf_predictions)
print('Confusion matrix\n\n', cm)

Confusion matrix

 [[425  50  59   8]
 [ 29 392  24  97]
 [ 74  24 425  20]
 [ 10  74  16 442]]


In [49]:
print('Training set score: {:.4f}'.format(rf.score(X_train, y_train)))

print('Test set score: {:.4f}'.format(rf.score(X_test, y_test)))

Training set score: 1.0000
Test set score: 0.7764


In [50]:
rf

RandomForestClassifier(n_estimators=300)

In [51]:
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
encoder = LabelEncoder()
encoder.fit(y)
encoded_Y = encoder.transform(y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

In [52]:
dummy_y

array([[1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       ...,
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.]], dtype=float32)

In [53]:
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(100, input_dim=8,kernel_initializer = 'uniform',  activation='relu'))
    model.add(Dense(100,kernel_initializer = 'uniform',  activation='relu'))
    model.add(Dense(200,kernel_initializer = 'uniform',  activation='relu'))

    model.add(Dense(4, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model



In [54]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=2000, batch_size=128, verbose=1)
kfold = KFold(n_splits=5, shuffle=True)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Epoch 1/2000
46/46 [==============================] - 0s 1ms/step - loss: 1.1888 - accuracy: 0.4775
Epoch 2/2000
46/46 [==============================] - 0s 1ms/step - loss: 0.9802 - accuracy: 0.5178
Epoch 3/2000
46/46 [==============================] - 0s 1ms/step - loss: 0.9511 - accuracy: 0.5429
Epoch 4/2000
46/46 [==============================] - 0s 1ms/step - loss: 0.9338 - accuracy: 0.5631
Epoch 5/2000
46/46 [==============================] - 0s 1ms/step - loss: 0.9253 - accuracy: 0.5666
Epoch 6/2000
46/46 [==============================] - 0s 1ms/step - loss: 0.9168 - accuracy: 0.5687
Epoch 7/2000
46/46 [==============================] - 0s 1ms/step - loss: 0.9063 - accuracy: 0.5721
Epoch 8/2000
46/46 [==============================] - 0s 1ms/step - loss: 0.9059 - accuracy: 0.5802
Epoch 9/2000
46/46 [==============================] - 0s 1ms/step - loss: 0.9063 - accuracy: 0.5673
Epoch 10/2000
46/46 [==============================] - 0s 1ms/step - loss: 0.8863 - accuracy: 0.5894

46/46 [==============================] - 0s 1ms/step - loss: 0.6655 - accuracy: 0.6866
Epoch 83/2000
46/46 [==============================] - 0s 1ms/step - loss: 0.6791 - accuracy: 0.6757
Epoch 84/2000
46/46 [==============================] - 0s 1ms/step - loss: 0.6651 - accuracy: 0.6870
Epoch 85/2000
46/46 [==============================] - 0s 1ms/step - loss: 0.6623 - accuracy: 0.6871
Epoch 86/2000
46/46 [==============================] - 0s 1ms/step - loss: 0.6678 - accuracy: 0.6844
Epoch 87/2000
46/46 [==============================] - 0s 1ms/step - loss: 0.6636 - accuracy: 0.6845
Epoch 88/2000
46/46 [==============================] - 0s 1ms/step - loss: 0.6665 - accuracy: 0.6921
Epoch 89/2000
46/46 [==============================] - 0s 1ms/step - loss: 0.6585 - accuracy: 0.6920
Epoch 90/2000
46/46 [==============================] - 0s 1ms/step - loss: 0.6623 - accuracy: 0.6913
Epoch 91/2000
46/46 [==============================] - 0s 1ms/step - loss: 0.6508 - accuracy: 0.6921
Epoc

46/46 [==============================] - 0s 1ms/step - loss: 0.5550 - accuracy: 0.7446
Epoch 163/2000
46/46 [==============================] - 0s 1ms/step - loss: 0.5593 - accuracy: 0.7430
Epoch 164/2000
46/46 [==============================] - 0s 1ms/step - loss: 0.5714 - accuracy: 0.7337
Epoch 165/2000
46/46 [==============================] - 0s 1ms/step - loss: 0.5566 - accuracy: 0.7404
Epoch 166/2000
46/46 [==============================] - 0s 1ms/step - loss: 0.5570 - accuracy: 0.7468
Epoch 167/2000
46/46 [==============================] - 0s 1ms/step - loss: 0.5603 - accuracy: 0.7411
Epoch 168/2000
46/46 [==============================] - 0s 1ms/step - loss: 0.5610 - accuracy: 0.7437
Epoch 169/2000
46/46 [==============================] - 0s 1ms/step - loss: 0.5583 - accuracy: 0.7421
Epoch 170/2000
46/46 [==============================] - 0s 1ms/step - loss: 0.5590 - accuracy: 0.7470
Epoch 171/2000
46/46 [==============================] - 0s 1ms/step - loss: 0.5486 - accuracy: 0.

46/46 [==============================] - 0s 1ms/step - loss: 0.4829 - accuracy: 0.7819
Epoch 243/2000
46/46 [==============================] - 0s 1ms/step - loss: 0.4849 - accuracy: 0.7831
Epoch 244/2000
46/46 [==============================] - 0s 1ms/step - loss: 0.4775 - accuracy: 0.7835
Epoch 245/2000
46/46 [==============================] - 0s 1ms/step - loss: 0.4832 - accuracy: 0.7824
Epoch 246/2000
46/46 [==============================] - 0s 1ms/step - loss: 0.4996 - accuracy: 0.7721
Epoch 247/2000
46/46 [==============================] - 0s 1ms/step - loss: 0.4844 - accuracy: 0.7810
Epoch 248/2000
46/46 [==============================] - 0s 1ms/step - loss: 0.4816 - accuracy: 0.7888
Epoch 249/2000
46/46 [==============================] - 0s 1ms/step - loss: 0.4727 - accuracy: 0.7899
Epoch 250/2000
46/46 [==============================] - 0s 1ms/step - loss: 0.4926 - accuracy: 0.7774
Epoch 251/2000
46/46 [==============================] - 0s 1ms/step - loss: 0.4877 - accuracy: 0.

46/46 [==============================] - 0s 1ms/step - loss: 0.4557 - accuracy: 0.7897
Epoch 323/2000
46/46 [==============================] - 0s 1ms/step - loss: 0.4381 - accuracy: 0.8070
Epoch 324/2000
46/46 [==============================] - 0s 1ms/step - loss: 0.4299 - accuracy: 0.8058
Epoch 325/2000
46/46 [==============================] - 0s 1ms/step - loss: 0.4290 - accuracy: 0.8104
Epoch 326/2000
46/46 [==============================] - 0s 1ms/step - loss: 0.4388 - accuracy: 0.8130
Epoch 327/2000
46/46 [==============================] - 0s 1ms/step - loss: 0.4351 - accuracy: 0.8122
Epoch 328/2000
46/46 [==============================] - 0s 1ms/step - loss: 0.4428 - accuracy: 0.8034
Epoch 329/2000
46/46 [==============================] - 0s 1ms/step - loss: 0.4424 - accuracy: 0.8013
Epoch 330/2000
46/46 [==============================] - 0s 1ms/step - loss: 0.4352 - accuracy: 0.8030
Epoch 331/2000
46/46 [==============================] - 0s 1ms/step - loss: 0.4382 - accuracy: 0.

...

46/46 [==============================] - 0s 2ms/step - loss: 0.2109 - accuracy: 0.9113
Epoch 1624/2000
46/46 [==============================] - 0s 2ms/step - loss: 0.1972 - accuracy: 0.9137
Epoch 1625/2000
46/46 [==============================] - 0s 2ms/step - loss: 0.2046 - accuracy: 0.9108
Epoch 1626/2000
46/46 [==============================] - 0s 2ms/step - loss: 0.2029 - accuracy: 0.9123
Epoch 1627/2000
46/46 [==============================] - 0s 2ms/step - loss: 0.2155 - accuracy: 0.9094
Epoch 1628/2000
46/46 [==============================] - 0s 2ms/step - loss: 0.1963 - accuracy: 0.9191
Epoch 1629/2000
46/46 [==============================] - 0s 2ms/step - loss: 0.2158 - accuracy: 0.9070
Epoch 1630/2000
46/46 [==============================] - 0s 2ms/step - loss: 0.2046 - accuracy: 0.9094
Epoch 1631/2000
46/46 [==============================] - 0s 2ms/step - loss: 0.1994 - accuracy: 0.9153
Epoch 1632/2000
46/46 [==============================] - 0s 2ms/step - loss: 0.2107 - acc

46/46 [==============================] - 0s 1ms/step - loss: 0.1855 - accuracy: 0.9220
Epoch 1703/2000
46/46 [==============================] - 0s 1ms/step - loss: 0.2103 - accuracy: 0.9115
Epoch 1704/2000
46/46 [==============================] - 0s 1ms/step - loss: 0.2128 - accuracy: 0.9101
Epoch 1705/2000
46/46 [==============================] - 0s 1ms/step - loss: 0.2118 - accuracy: 0.9089
Epoch 1706/2000
46/46 [==============================] - 0s 1ms/step - loss: 0.2351 - accuracy: 0.9030
Epoch 1707/2000
46/46 [==============================] - 0s 1ms/step - loss: 0.2028 - accuracy: 0.9153
Epoch 1708/2000
46/46 [==============================] - 0s 1ms/step - loss: 0.2060 - accuracy: 0.9142
Epoch 1709/2000
46/46 [==============================] - 0s 1ms/step - loss: 0.2183 - accuracy: 0.9096
Epoch 1710/2000
46/46 [==============================] - 0s 1ms/step - loss: 0.1930 - accuracy: 0.9148
Epoch 1711/2000
46/46 [==============================] - 0s 985us/step - loss: 0.1864 - a

46/46 [==============================] - 0s 1ms/step - loss: 0.2248 - accuracy: 0.9071
Epoch 1782/2000
46/46 [==============================] - 0s 1ms/step - loss: 0.1888 - accuracy: 0.9222
Epoch 1783/2000
46/46 [==============================] - 0s 1ms/step - loss: 0.1988 - accuracy: 0.9142
Epoch 1784/2000
46/46 [==============================] - 0s 1ms/step - loss: 0.1859 - accuracy: 0.9192
Epoch 1785/2000
46/46 [==============================] - 0s 1ms/step - loss: 0.2086 - accuracy: 0.9142
Epoch 1786/2000
46/46 [==============================] - 0s 1ms/step - loss: 0.2054 - accuracy: 0.9122
Epoch 1787/2000
46/46 [==============================] - 0s 1ms/step - loss: 0.1986 - accuracy: 0.9191
Epoch 1788/2000
46/46 [==============================] - 0s 1ms/step - loss: 0.2364 - accuracy: 0.9004
Epoch 1789/2000
46/46 [==============================] - 0s 2ms/step - loss: 0.2197 - accuracy: 0.9077
Epoch 1790/2000
46/46 [==============================] - 0s 1ms/step - loss: 0.2126 - acc

46/46 [==============================] - 0s 2ms/step - loss: 0.2064 - accuracy: 0.9139
Epoch 1861/2000
46/46 [==============================] - 0s 2ms/step - loss: 0.2016 - accuracy: 0.9161
Epoch 1862/2000
46/46 [==============================] - 0s 2ms/step - loss: 0.1953 - accuracy: 0.9211
Epoch 1863/2000
46/46 [==============================] - 0s 2ms/step - loss: 0.1855 - accuracy: 0.9225
Epoch 1864/2000
46/46 [==============================] - 0s 2ms/step - loss: 0.1907 - accuracy: 0.9205
Epoch 1865/2000
46/46 [==============================] - 0s 2ms/step - loss: 0.1906 - accuracy: 0.9167
Epoch 1866/2000
46/46 [==============================] - 0s 2ms/step - loss: 0.1823 - accuracy: 0.9286
Epoch 1867/2000
46/46 [==============================] - 0s 1ms/step - loss: 0.1783 - accuracy: 0.9262
Epoch 1868/2000
46/46 [==============================] - 0s 1ms/step - loss: 0.1808 - accuracy: 0.9286
Epoch 1869/2000
46/46 [==============================] - 0s 1ms/step - loss: 0.1853 - acc

46/46 [==============================] - 0s 2ms/step - loss: 0.1852 - accuracy: 0.9241
Epoch 1940/2000
46/46 [==============================] - 0s 2ms/step - loss: 0.1966 - accuracy: 0.9170
Epoch 1941/2000
46/46 [==============================] - 0s 2ms/step - loss: 0.1923 - accuracy: 0.9177
Epoch 1942/2000
46/46 [==============================] - 0s 2ms/step - loss: 0.1933 - accuracy: 0.9180
Epoch 1943/2000
46/46 [==============================] - 0s 2ms/step - loss: 0.1887 - accuracy: 0.9199
Epoch 1944/2000
46/46 [==============================] - 0s 2ms/step - loss: 0.2199 - accuracy: 0.9101
Epoch 1945/2000
46/46 [==============================] - 0s 2ms/step - loss: 0.2019 - accuracy: 0.9167
Epoch 1946/2000
46/46 [==============================] - 0s 2ms/step - loss: 0.1930 - accuracy: 0.9165
Epoch 1947/2000
46/46 [==============================] - 0s 2ms/step - loss: 0.2045 - accuracy: 0.9122
Epoch 1948/2000
46/46 [==============================] - 0s 2ms/step - loss: 0.1860 - acc

In [55]:
print(results)

[0.78284925 0.78354079 0.77731675 0.78685123 0.78200692]


In [56]:
# estimator.fit(X_train, y_train)
# predictions = estimator.predict(X_test)
#print(predictions)

In [57]:
# print(encoder.inverse_transform(predictions))

In [58]:
X_train, X_test, Y_train, Y_test = train_test_split(X, dummy_y, test_size=0.3)
estimator.fit(X_train, Y_train)
predictions = estimator.predict(X_test)

Epoch 1/2000
40/40 [==============================] - 0s 896us/step - loss: 1.2017 - accuracy: 0.4540
Epoch 2/2000
40/40 [==============================] - 0s 839us/step - loss: 0.9938 - accuracy: 0.5098
Epoch 3/2000
40/40 [==============================] - 0s 835us/step - loss: 0.9598 - accuracy: 0.5363
Epoch 4/2000
40/40 [==============================] - 0s 825us/step - loss: 0.9396 - accuracy: 0.5604
Epoch 5/2000
40/40 [==============================] - 0s 819us/step - loss: 0.9293 - accuracy: 0.5588
Epoch 6/2000
40/40 [==============================] - 0s 819us/step - loss: 0.9192 - accuracy: 0.5722
Epoch 7/2000
40/40 [==============================] - 0s 830us/step - loss: 0.9122 - accuracy: 0.5756
Epoch 8/2000
40/40 [==============================] - 0s 820us/step - loss: 0.9068 - accuracy: 0.5695
Epoch 9/2000
40/40 [==============================] - 0s 817us/step - loss: 0.8973 - accuracy: 0.5835
Epoch 10/2000
40/40 [==============================] - 0s 1ms/step - loss: 0.9009 

40/40 [==============================] - 0s 920us/step - loss: 0.6899 - accuracy: 0.6762
Epoch 82/2000
40/40 [==============================] - 0s 910us/step - loss: 0.6772 - accuracy: 0.6829
Epoch 83/2000
40/40 [==============================] - 0s 906us/step - loss: 0.6728 - accuracy: 0.6885
Epoch 84/2000
40/40 [==============================] - 0s 919us/step - loss: 0.6717 - accuracy: 0.6825
Epoch 85/2000
40/40 [==============================] - 0s 909us/step - loss: 0.6692 - accuracy: 0.6831
Epoch 86/2000
40/40 [==============================] - 0s 911us/step - loss: 0.6672 - accuracy: 0.6883
Epoch 87/2000
40/40 [==============================] - 0s 921us/step - loss: 0.6711 - accuracy: 0.6853
Epoch 88/2000
40/40 [==============================] - 0s 905us/step - loss: 0.6710 - accuracy: 0.6881
Epoch 89/2000
40/40 [==============================] - 0s 913us/step - loss: 0.6666 - accuracy: 0.6867
Epoch 90/2000
40/40 [==============================] - 0s 905us/step - loss: 0.6589 - a

40/40 [==============================] - 0s 1ms/step - loss: 0.5659 - accuracy: 0.7446
Epoch 162/2000
40/40 [==============================] - 0s 1ms/step - loss: 0.5597 - accuracy: 0.7492
Epoch 163/2000
40/40 [==============================] - 0s 1ms/step - loss: 0.5694 - accuracy: 0.7472
Epoch 164/2000
40/40 [==============================] - 0s 1ms/step - loss: 0.5587 - accuracy: 0.7428
Epoch 165/2000
40/40 [==============================] - 0s 1ms/step - loss: 0.5579 - accuracy: 0.7466
Epoch 166/2000
40/40 [==============================] - 0s 1ms/step - loss: 0.5658 - accuracy: 0.7424
Epoch 167/2000
40/40 [==============================] - 0s 2ms/step - loss: 0.5530 - accuracy: 0.7484
Epoch 168/2000
40/40 [==============================] - 0s 2ms/step - loss: 0.5748 - accuracy: 0.7383
Epoch 169/2000
40/40 [==============================] - 0s 2ms/step - loss: 0.5498 - accuracy: 0.7545
Epoch 170/2000
40/40 [==============================] - 0s 2ms/step - loss: 0.5565 - accuracy: 0.

40/40 [==============================] - 0s 2ms/step - loss: 0.4903 - accuracy: 0.7784
Epoch 242/2000
40/40 [==============================] - 0s 2ms/step - loss: 0.4944 - accuracy: 0.7792
Epoch 243/2000
40/40 [==============================] - 0s 2ms/step - loss: 0.4890 - accuracy: 0.7784
Epoch 244/2000
40/40 [==============================] - 0s 2ms/step - loss: 0.4919 - accuracy: 0.7753
Epoch 245/2000
40/40 [==============================] - 0s 2ms/step - loss: 0.5010 - accuracy: 0.7721
Epoch 246/2000
40/40 [==============================] - 0s 2ms/step - loss: 0.4982 - accuracy: 0.7747
Epoch 247/2000
40/40 [==============================] - 0s 2ms/step - loss: 0.4978 - accuracy: 0.7733
Epoch 248/2000
40/40 [==============================] - 0s 2ms/step - loss: 0.5158 - accuracy: 0.7664
Epoch 249/2000
40/40 [==============================] - 0s 2ms/step - loss: 0.4808 - accuracy: 0.7838
Epoch 250/2000
40/40 [==============================] - 0s 2ms/step - loss: 0.4904 - accuracy: 0.

40/40 [==============================] - 0s 2ms/step - loss: 0.4558 - accuracy: 0.7970
Epoch 322/2000
40/40 [==============================] - 0s 2ms/step - loss: 0.4432 - accuracy: 0.8071
Epoch 323/2000
40/40 [==============================] - 0s 2ms/step - loss: 0.4380 - accuracy: 0.8015
Epoch 324/2000
40/40 [==============================] - 0s 2ms/step - loss: 0.4431 - accuracy: 0.8021
Epoch 325/2000
40/40 [==============================] - 0s 2ms/step - loss: 0.4376 - accuracy: 0.8041
Epoch 326/2000
40/40 [==============================] - 0s 2ms/step - loss: 0.4548 - accuracy: 0.7956
Epoch 327/2000
40/40 [==============================] - 0s 2ms/step - loss: 0.4328 - accuracy: 0.8085
Epoch 328/2000
40/40 [==============================] - 0s 2ms/step - loss: 0.4340 - accuracy: 0.8136
Epoch 329/2000
40/40 [==============================] - 0s 2ms/step - loss: 0.4425 - accuracy: 0.8027
Epoch 330/2000
40/40 [==============================] - 0s 2ms/step - loss: 0.4395 - accuracy: 0.

...

40/40 [==============================] - 0s 2ms/step - loss: 0.2064 - accuracy: 0.9103
Epoch 1673/2000
40/40 [==============================] - 0s 2ms/step - loss: 0.2235 - accuracy: 0.9033
Epoch 1674/2000
40/40 [==============================] - 0s 2ms/step - loss: 0.2106 - accuracy: 0.9069
Epoch 1675/2000
40/40 [==============================] - 0s 2ms/step - loss: 0.1987 - accuracy: 0.9140
Epoch 1676/2000
40/40 [==============================] - 0s 2ms/step - loss: 0.2199 - accuracy: 0.9091
Epoch 1677/2000
40/40 [==============================] - 0s 2ms/step - loss: 0.2130 - accuracy: 0.9049
Epoch 1678/2000
40/40 [==============================] - 0s 2ms/step - loss: 0.2058 - accuracy: 0.9138
Epoch 1679/2000
40/40 [==============================] - 0s 2ms/step - loss: 0.2132 - accuracy: 0.9049
Epoch 1680/2000
40/40 [==============================] - 0s 2ms/step - loss: 0.1958 - accuracy: 0.9186
Epoch 1681/2000
40/40 [==============================] - 0s 2ms/step - loss: 0.2154 - acc

40/40 [==============================] - 0s 2ms/step - loss: 0.2026 - accuracy: 0.9130
Epoch 1752/2000
40/40 [==============================] - 0s 2ms/step - loss: 0.2169 - accuracy: 0.9073
Epoch 1753/2000
40/40 [==============================] - 0s 2ms/step - loss: 0.2058 - accuracy: 0.9122
Epoch 1754/2000
40/40 [==============================] - 0s 2ms/step - loss: 0.2148 - accuracy: 0.9063
Epoch 1755/2000
40/40 [==============================] - 0s 2ms/step - loss: 0.1920 - accuracy: 0.9186
Epoch 1756/2000
40/40 [==============================] - 0s 2ms/step - loss: 0.2075 - accuracy: 0.9101
Epoch 1757/2000
40/40 [==============================] - 0s 2ms/step - loss: 0.2021 - accuracy: 0.9136
Epoch 1758/2000
40/40 [==============================] - 0s 2ms/step - loss: 0.2091 - accuracy: 0.9099
Epoch 1759/2000
40/40 [==============================] - 0s 2ms/step - loss: 0.1987 - accuracy: 0.9132
Epoch 1760/2000
40/40 [==============================] - 0s 2ms/step - loss: 0.1895 - acc

40/40 [==============================] - 0s 2ms/step - loss: 0.1845 - accuracy: 0.9255
Epoch 1831/2000
40/40 [==============================] - 0s 2ms/step - loss: 0.1936 - accuracy: 0.9180
Epoch 1832/2000
40/40 [==============================] - 0s 2ms/step - loss: 0.1838 - accuracy: 0.9245
Epoch 1833/2000
40/40 [==============================] - 0s 2ms/step - loss: 0.1952 - accuracy: 0.9184
Epoch 1834/2000
40/40 [==============================] - 0s 2ms/step - loss: 0.1897 - accuracy: 0.9162
Epoch 1835/2000
40/40 [==============================] - 0s 2ms/step - loss: 0.1879 - accuracy: 0.9205
Epoch 1836/2000
40/40 [==============================] - 0s 2ms/step - loss: 0.2085 - accuracy: 0.9118
Epoch 1837/2000
40/40 [==============================] - 0s 2ms/step - loss: 0.1962 - accuracy: 0.9166
Epoch 1838/2000
40/40 [==============================] - 0s 2ms/step - loss: 0.1899 - accuracy: 0.9184
Epoch 1839/2000
40/40 [==============================] - 0s 2ms/step - loss: 0.2092 - acc

40/40 [==============================] - 0s 2ms/step - loss: 0.1926 - accuracy: 0.9186
Epoch 1910/2000
40/40 [==============================] - 0s 2ms/step - loss: 0.1786 - accuracy: 0.9253
Epoch 1911/2000
40/40 [==============================] - 0s 2ms/step - loss: 0.1839 - accuracy: 0.9180
Epoch 1912/2000
40/40 [==============================] - 0s 2ms/step - loss: 0.1934 - accuracy: 0.9194
Epoch 1913/2000
40/40 [==============================] - 0s 2ms/step - loss: 0.2017 - accuracy: 0.9152
Epoch 1914/2000
40/40 [==============================] - 0s 2ms/step - loss: 0.2033 - accuracy: 0.9160
Epoch 1915/2000
40/40 [==============================] - 0s 2ms/step - loss: 0.1877 - accuracy: 0.9207
Epoch 1916/2000
40/40 [==============================] - 0s 2ms/step - loss: 0.1926 - accuracy: 0.9156
Epoch 1917/2000
40/40 [==============================] - 0s 2ms/step - loss: 0.1837 - accuracy: 0.9223
Epoch 1918/2000
40/40 [==============================] - 0s 2ms/step - loss: 0.2076 - acc

40/40 [==============================] - 0s 1ms/step - loss: 0.1863 - accuracy: 0.9211
Epoch 1989/2000
40/40 [==============================] - 0s 1ms/step - loss: 0.1876 - accuracy: 0.9184
Epoch 1990/2000
40/40 [==============================] - 0s 1ms/step - loss: 0.1835 - accuracy: 0.9223
Epoch 1991/2000
40/40 [==============================] - 0s 1ms/step - loss: 0.1938 - accuracy: 0.9158
Epoch 1992/2000
40/40 [==============================] - 0s 1ms/step - loss: 0.1812 - accuracy: 0.9227
Epoch 1993/2000
40/40 [==============================] - 0s 1ms/step - loss: 0.1907 - accuracy: 0.9195
Epoch 1994/2000
40/40 [==============================] - 0s 1ms/step - loss: 0.2119 - accuracy: 0.9097
Epoch 1995/2000
40/40 [==============================] - 0s 1ms/step - loss: 0.2059 - accuracy: 0.9114
Epoch 1996/2000
40/40 [==============================] - 0s 1ms/step - loss: 0.2090 - accuracy: 0.9148
Epoch 1997/2000
40/40 [==============================] - 0s 1ms/step - loss: 0.2017 - acc

In [59]:
print(predictions[0])
print(encoder.inverse_transform(predictions))

1
['Excellent' 'Fair' 'Good' ... 'Fair' 'Fair' 'Excellent']


In [60]:
Y_test[0]

array([0., 1., 0., 0.], dtype=float32)

In [61]:
import numpy as np
rounded_labels=np.argmax(Y_test, axis=1)
rounded_labels[3]
# 2

2

In [62]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(rounded_labels, predictions)
cm

array([[363,  56,  99,  12],
       [ 44, 433,  21,  76],
       [ 66,  18, 420,  10],
       [ 14,  84,  13, 440]])

In [63]:
print(classification_report(rounded_labels, predictions))

              precision    recall  f1-score   support

           0       0.75      0.68      0.71       530
           1       0.73      0.75      0.74       574
           2       0.76      0.82      0.79       514
           3       0.82      0.80      0.81       551

    accuracy                           0.76      2169
   macro avg       0.76      0.76      0.76      2169
weighted avg       0.76      0.76      0.76      2169



In [64]:
accuracy_score(rounded_labels, predictions)
print('Accuracy is ' + str(round(accuracy_score(rounded_labels, predictions),2)))

Accuracy is 0.76


In [65]:
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

In [66]:
cmn= np.around(cm / cm.astype(np.float).sum(axis=1, keepdims=True),2) 

In [67]:
cmn

array([[0.68, 0.11, 0.19, 0.02],
       [0.08, 0.75, 0.04, 0.13],
       [0.13, 0.04, 0.82, 0.02],
       [0.03, 0.15, 0.02, 0.8 ]])